## Solving a simple non-linear optimization problem with open source solver in Python

#### The Problem

maximize:  $\sum_{j}(X_{j}-cX_{j}^2)ROI_{j}$ 

subject to:  $\sum_{j}X_{j} \leq B$

subject to:  $X_j \leq S_j \space\space\space \forall j$

subject to:  $X_{TV} \leq 0.5\sum_j X_j$

$j \in J = Index(Marketing Channel)$ 

$X_j = Decision Variable(How\space much \space to \space invest \space in \space each \space channel \space j)$

$B = Parameter(Budget)$

$S_j = Parameter(Channel \space satuarion \space point)$

$ROI_j = Parameter(ReturnOnInvestment)$

#### DATA: 

In [58]:
# Assume the following data

Chanell = ["FB","INS","TV","GOOGL"]
B = 100000
ROI = {"FB":0.08,"INS":0.1,"TV":0.15,"GOOGL":0.07}
S = {"FB":90000,"INS":70000,"TV":80000,"GOOGL":20000}
c = 0.0002

In [59]:
# IMPORT PACKAGES
import pulp as plp
import numpy as np

In [35]:
# Write the model in a function
def MIO_non_linear(Chanell,B,ROI,S):
    
    # Define the type of problem
    prob = plp.LpProblem('MIO',plp.LpMaximize)
    
    # Define the decision variable
    X = plp.LpVariable.dicts("X", ((j) for j in Chanell), lowBound=0,cat="Continuous")
    
    # Define the model obj. function
    prob += plp.lpSum([(X[j]*ROI[j]) for j in Chanell])
    
    # Define the model constraints
    prob += plp.lpSum([X[j] for j in Chanell]) <= B
    
    prob += plp.lpSum([X[j] for j in Chanell])*0.5 >= X["TV"]
    
    for j in Chanell:
        prob += X[j] <= S[j]
    
    # Model Solve
    prob.solve()
    print(plp.LpStatus[prob.status])
    
    # Structure the output
    the_X = np.array([X[j].varValue for j in Chanell])
    
    profit = 0
    for i in range(len(the_X)):
        profit += the_X[i]*list(ROI.values())[i]
        
    # Return values
    return the_X, profit
    

In [36]:
# Solve the model, given the data, and examine the results 

MIO_non_linear(Chanell,B,ROI,S)

Optimal


(array([    0., 50000., 50000.,     0.]), 12500.0)

### CONCLUSION: PuLP cannot solve Non-linear problems

### Solving same problem but in Pyomo

In [60]:
import pyomo.environ as pyo

In [8]:
# model formulation
model = pyo.ConcreteModel()

# variables
model.x = pyo.Var(Chanell,domain=pyo.NonNegativeReals)

# objective
profit = sum(ROI[j] * model.x[j] for j in Chanell)
model.objective = pyo.Objective(expr = profit, sense=pyo.maximize)

# constraints
model.constraints = pyo.ConstraintList()
for j in Chanell:
    model.constraints.add(model.x[j] <= S[j])
model.constraints.add(B >= sum(model.x[j] for j in Chanell))
model.constraints.add(model.x["TV"] <= 0.5*sum(model.x[j] for j in Chanell))

In [39]:
solver = pyo.SolverFactory('glpk')
solver.solve(model)
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {1, 2, 3, 4, 5, 6}
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'FB', 'INS', 'TV', 'GOOGL'}

1 Var Declarations
    x : Size=4, Index=x_index
        Key   : Lower : Value   : Upper : Fixed : Stale : Domain
           FB :     0 :     0.0 :  None : False : False : NonNegativeReals
        GOOGL :     0 :     0.0 :  None : False : False : NonNegativeReals
          INS :     0 : 50000.0 :  None : False : False : NonNegativeReals
           TV :     0 : 50000.0 :  None : False : False : NonNegativeReals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 0.08*x[FB] + 0.1*x[INS] + 0.15*x[TV] + 0.07*x[GOOGL]

1 Constraint Declaratio

In [11]:
profit()

12500.0

In [12]:
print({j: model.x[j]() for j in Chanell})

{'FB': 0.0, 'INS': 50000.0, 'TV': 50000.0, 'GOOGL': 0.0}


### Now Non-linear

In [68]:
# model formulation
model = pyo.ConcreteModel()

# variables
model.x = pyo.Var(Chanell,domain=pyo.NonNegativeReals)
x = model.x
# objective
profit = sum(ROI[j]*(x[j] - c*x[j]**2) for j in Chanell)
model.objective = pyo.Objective(expr = profit, sense=maximize)

# constraints
model.constraints = pyo.ConstraintList()
for j in Chanell:
    model.constraints.add(model.x[j] - S[j] <= 0)
model.constraints.add(B - sum(x[j] for j in Chanell) >= 0)
for j in Chanell:
    model.constraints.add(model.x[j] >= 0)
model.constraints.add(model.x["TV"] - 0.2*sum(model.x[j] for j in Chanell) <= 0)

In [63]:
solver = pyo.SolverFactory('baron.exe',executable = "C:\\Users\\MyName\\AMPL Files\\ampl.mswin64\\baron.exe")
solver.solve(model)
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'FB', 'INS', 'TV', 'GOOGL'}

1 Var Declarations
    x : Size=4, Index=x_index
        Key   : Lower : Value              : Upper : Fixed : Stale : Domain
           FB :     0 : 2717.8423236514523 :  None : False : False : NonNegativeReals
        GOOGL :     0 : 2748.9626556016597 :  None : False : False : NonNegativeReals
          INS :     0 : 2674.2738589211617 :  None : False : False : NonNegativeReals
           TV :     0 : 2035.2697095435685 :  None : False : False : NonNegativeReals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 0.08*(x[F

#### Notes: The solver was suppose to be able to solve the problem and to be in the same folder or use "executable = 'path' "
#### Baron solver is a commercial solver 

In [67]:
from pyomo.opt import SolverFactory
from pyomo.environ import *
solver = SolverFactory('couenne.exe', executable = "C:\\Users\\MyName\\Solver\\COIN-OR-1.8.0-win32-msvc12\\COIN-OR\\win32-msvc12\\bin\\couenne.exe")
solver.solve(model)
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'FB', 'INS', 'TV', 'GOOGL'}

1 Var Declarations
    x : Size=4, Index=x_index
        Key   : Lower : Value              : Upper : Fixed : Stale : Domain
           FB :     0 :  2717.842323647428 :  None : False : False : NonNegativeReals
        GOOGL :     0 : 2748.9626555970412 :  None : False : False : NonNegativeReals
          INS :     0 :  2674.273858917943 :  None : False : False : NonNegativeReals
           TV :     0 : 2035.2697095527444 :  None : False : False : NonNegativeReals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 0.08*(x[F

In [69]:
opt = SolverFactory("ipopt", executable = "C:\\Users\\MyName\\Solver\\COIN-OR-1.8.0-win32-msvc12\\COIN-OR\\win32-msvc12\\bin\\ipopt.exe")
opt.solve(model)
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'FB', 'INS', 'TV', 'GOOGL'}

1 Var Declarations
    x : Size=4, Index=x_index
        Key   : Lower : Value              : Upper : Fixed : Stale : Domain
           FB :     0 :  2717.842323734292 :  None : False : False : NonNegativeReals
        GOOGL :     0 :  2748.962655691427 :  None : False : False : NonNegativeReals
          INS :     0 : 2674.2738589879723 :  None : False : False : NonNegativeReals
           TV :     0 : 2035.2697095260462 :  None : False : False : NonNegativeReals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 0.08*(x[F

In [70]:
model2 = pyo.ConcreteModel()
model2.x = pyo.Var(bounds= (-5,5))
model2.y = pyo.Var(bounds= (-5,5))
x = model2.x
y = model2.y
model2.obj = pyo.Objective(expr=cos(x+1)+cos(x)*cos(y), sense = maximize)
opt = SolverFactory('couenne',executable = "C:\\Users\\MyName\\Solver\\COIN-OR-1.8.0-win32-msvc12\\COIN-OR\\win32-msvc12\\bin\\couenne.exe")
opt.solve(model2)
model2.pprint()
print(pyo.value(x))
print(pyo.value(y))

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value               : Upper : Fixed : Stale : Domain
        None :    -5 : -0.4999999999972364 :     5 : False : False :  Reals
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :    -5 :   0.0 :     5 : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : cos(x + 1) + cos(x)*cos(y)

3 Declarations: x y obj
-0.4999999999972364
0.0


##### Note: couenne and ipopt cannot deal with powers of >1 & <2 or similar non-integer, but baron can!